In [96]:
import pymssql
from dotenv import load_dotenv
import os
load_dotenv()

def connect_db():
    server = os.getenv("SERVER")
    username = os.getenv("USERNAME")
    password = os.getenv("PASSWORD")
    database = os.getenv("DATABASE")
    port = os.getenv("PORT")
    uri = f"mssql+pymssql://{username}:{password}@{server}/{database}"
    return pymssql.connect(server, username, password, database), uri

conn,uri = connect_db()
print('Conectado ao banco de dados')

Conectado ao banco de dados


In [97]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine
# inspector
from sqlalchemy import inspect

engine=create_engine(uri)

In [98]:
inspector = inspect(engine)

# Obtenha os nomes das tabela

In [99]:
tabelas = inspector.get_table_names()

In [100]:
# # registre em um txt no formato: A tabela <nome da tabela> tem as colunas: <nome da coluna> <nome da coluna> <nome da coluna> ... e pule uma linha
# with open('db_schema.txt', 'w') as f:
#     for tabela in tabelas:
#         f.write(f'table {tabela} cols: ')
#         for coluna in inspector.get_columns(tabela):
#             f.write(f'{coluna["name"]} ')
#         f.write('\n')
#         print(f'Escrevendo {tabela} no arquivo')
    

In [101]:
# count tokens
tokens = 0
db_schema = open('db_schema.txt', 'r')

for line in db_schema:
    tokens += len(line.split())
print(f'Número de tokens: {tokens}')

Número de tokens: 5123


In [102]:
# print lines 
db_schema = open('db_schema.txt', 'r')

In [103]:
# # Caching the prompt
# from langchain.cache import SQLiteCache
# from langchain.globals import set_llm_cache
# from langchain.prompts import PromptTemplate

# question = "Quantos os projetos tem carteira para setembro de 2021?"
# prompt_template = PromptTemplate.from_template(template=template)
# prompt = prompt_template.format(db_schema_list=db_schema_list,question=question)

In [104]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    length_function=len
)

In [105]:
with open('db_schema.txt') as f:
   db_schema = f.read() 
chunks = splitter.create_documents([db_schema])

In [106]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
set_llm_cache(SQLiteCache(database_path='./cache/.langchain.db'))

In [107]:
def print_embedding_cost(text):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in text])
    print(f"Total tokens: {total_tokens}")
    print('Embedding cost in USD:',total_tokens/1000 * 0.0004)

In [108]:
print_embedding_cost(chunks)

Total tokens: 18356
Embedding cost in USD: 0.007342400000000001


In [109]:
# for i in pc.list_indexes().names():
#     print('Deletando indice ', i)
#     pc.delete_index(i)

In [110]:
import pinecone
from langchain_community.vectorstores import Pinecone

pc = pinecone.Pinecone()

index_name = 'geoex-sql-embeddings'
if index_name not in pc.list_indexes().names():
    print('Criando index',index_name)
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=pinecone.PodSpec(
            environment='gcp-starter'
        )
        
    )
    print('done')

Criando VectorStore dos metadados do Banco

In [111]:
vector_store = Pinecone.from_documents(chunks,embeddings,index_name=index_name)

In [112]:
pc.Index(index_name).describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00652,
 'namespaces': {'': {'vector_count': 652}},
 'total_vector_count': 652}

In [113]:
query = 'Projeto Normal'
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='ProjetoPEPId'), Document(page_content='ProjetoPEPId'), Document(page_content='PlanejamentoTipoObraId CustoEstimado'), Document(page_content='PlanejamentoTipoObraId CustoEstimado')]


In [114]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0)

In [152]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':50})
chain = RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=retriever)

In [156]:
prompt_template = """Voce é um assistente chamado GeoAI que so responde em formato de script de
    consulta em SQL para SQL server, voce so faz comandos de SELECT e Nunca de Alteração dos dados ou metadados. Baseado nos dados fornecidos, monte a melhor consulta 
    para responder esta pergunta de forma mais ao pé da letra possível: '{question}'
    """
    
question = "Quero informacoes sobre o projeto que tem a nota numero 9200848541"
prompt_template = PromptTemplate.from_template(template=prompt_template)
prompt = prompt_template.format(question=question)

In [157]:
prompt

"Voce é um assistente chamado GeoAI que so responde em formato de script de\n    consulta em SQL para SQL server, voce so faz comandos de SELECT e Nunca de Alteração dos dados ou metadados. Baseado nos dados fornecidos, monte a melhor consulta \n    para responder esta pergunta de forma mais ao pé da letra possível: 'Quero informacoes sobre o projeto que tem a nota numero 9200848541'\n    se precisar de mais informacoes ou ficar com dúvida de qual tabela usar, pergunte ao usuário informando as opcoes"

In [158]:
resp = chain.run(prompt)
print(resp)

```sql
SELECT *
FROM ProjetoEletrobrasUC
WHERE Numero = 9200848541;
```
